## Define a model
Take it easy and start with a simple decision tree classifier.

In [21]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:

# Load iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree accuracy on Iris dataset: {accuracy:.2f}")

Decision Tree accuracy on Iris dataset: 1.00


In [2]:
import pickle

with open('model/decision_tree_clf.pkl', 'wb') as f:
    pickle.dump(clf, f)

## Try prediction

In [25]:
# In a new Jupyter cell

import requests
import json

# --- 1. Define the API Endpoint and Data ---

# The URL of the API endpoint running in your Docker container
# Remember we mapped port 5001 on our host to port 5000 in the container
api_url = "http://localhost:5001/predict"

# The data you want to send for prediction.
# The structure MUST match what your Flask app expects: a dictionary with a "features" key.
# Let's say your model was trained on 4 features (e.g., Iris dataset).
test_data = {
    "features": [5.1, 3.5, 1.4, 0.2]  # Example: Sepal Length, Sepal Width, Petal Length, Petal Width
}


# --- 2. Send the POST Request ---

# The requests.post() function sends an HTTP POST request.
# The `json` parameter automatically does two things:
#   1. It serializes your Python dictionary `test_data` into a JSON string.
#   2. It sets the `Content-Type` header to `application/json`.
try:
    response = requests.post(api_url, json=test_data)
    response.raise_for_status()  # This will raise an exception for bad status codes (4xx or 5xx)

    # --- 3. Process the Response ---

    # The server responds with JSON, so we can parse it using the .json() method
    prediction_result = response.json()

    print(f"✅ Success! API call was successful.")
    print("-" * 30)
    print(f"   Input data: {test_data['features']}")
    print(f"   Prediction received: {prediction_result['prediction']}")
    print("-" * 30)


except requests.exceptions.ConnectionError as e:
    print(f"❌ Error: Could not connect to the API.")
    print(f"   Please ensure the Docker container is running and the URL '{api_url}' is correct.")

except requests.exceptions.HTTPError as e:
    print(f"❌ Error: An HTTP error occurred.")
    print(f"   Status Code: {e.response.status_code}")
    print(f"   Response Body: {e.response.text}") # Show the error message from the server

except Exception as e:
    print(f"An unexpected error occurred: {e}")

✅ Success! API call was successful.
------------------------------
   Input data: [5.1, 3.5, 1.4, 0.2]
   Prediction received: [0]
------------------------------


In [26]:
prediction_result

{'class_name': 'setosa',
 'features': [5.1, 3.5, 1.4, 0.2],
 'model_version': 'Decision Tree Classifier v1.0',
 'prediction': [0]}